In [184]:
from lxml import etree
import requests

BASE_DOMAIN = 'http://www.dytt8.net/'
HEADERS = {
    "User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
    "Host": 'www.dytt8.net',
    'Upgrade-Insecure-Requests': "1",
    "Connection": "keep-alive",
    "Pragma": 'no-cache',
    "Cookie": 'XLA_CI=1706c582df1bcaf44a14616471c690f9; UM_distinctid=165a0615b8d4b0-028660a7cc3cf4-34677908-fa000-165a0615b8f812; 37cs_user=37cs92189309021; 37cs_pidx=4; 37cs_show=200%2C201%2C76%2C77; CNZZDATA1260535040=299546897-1535995289-%7C1536000273; cscpvrich5041_fidx=2'
}

def get_detail_urls(url):
    response = requests.get(url, headers=HEADERS)
    text = response.text 
    #乱码也无所谓，因为我们只想要得到每个电影的detail_url
    #但是如果用response.content.decode('gbk')，有些元素不能解码，会出错
    #如果有需要解析一些内容，就先用response.text.encode('gbk')
    #再decode自己需要的内容。不然的话，如果decode整个网页，会出错，解析不了。
    html = etree.HTML(text)
    detail_urls = html.xpath("//table[@class='tbspan']//a/@href")
    detail_urls = map(lambda url: BASE_DOMAIN + url, detail_urls)
    # 这个匿名函数等价于
    '''
    def abc(url):
        return BASE_DOMAIN + url
    index = 0
    for detail_url in detail_urls:
        detail_url = abc(detail_url)
        detail_urls[index] = detail_url
        index += 1
    '''
    return detail_urls

def parse_detail_page(url):
    movie = {}
    response = requests.get(url, headers=HEADERS)
    text = response.content.decode('gbk')
    html = etree.HTML(text)
    title = html.xpath("//div[@class='title_all']//font[@color='#07519a']/text()")[0]
    movie['title'] = title
    
    #Zoom包含了我们后面需要的电影信息
    zoomE = html.xpath("//div[@id='Zoom']")[0]
    #.//在当前元素下查找
    imgs = zoomE.xpath(".//img/@src")
    if len(imgs) == 2: #有时候只有照片，没有截图
        cover = imgs[0]
        screenshot = imgs[1]
        movie['cover'] = cover
        movie['screenshot'] = screenshot  
    
    def parse_info(info, rule):
        return info.replace(rule, "").strip()
    
    infos = zoomE.xpath(".//text()")
    for index,info in enumerate(infos):
        if info.startswith("◎年　　代"):
            info = parse_info(info, "◎年　　代")
            movie['year'] = info
        elif info.startswith("◎产　　地"):
            info = parse_info(info, "◎产　　地")
            movie['country'] = info
        elif info.startswith("◎类　　别"):
            info = parse_info(info, "◎类　　别")
            movie['category'] = info
        elif info.startswith("◎豆瓣评分"):
            info = parse_info(info, "◎豆瓣评分")
            movie['douban_rating'] = info
        elif info.startswith("◎IMDb评分"):
            info = parse_info(info, "◎IMDb评分")
            movie['imdb_rating'] = info
        elif info.startswith("◎片　　长"):
            info = parse_info(info, "◎片　　长")
            movie['duration'] = info
        elif info.startswith("◎导　　演"):
            info = parse_info(info, "◎导　　演")
            movie['director'] = info
        elif info.startswith("◎主　　演"):
            info = parse_info(info, "◎主　　演")
            # info已经包含了第一个演员
            actors = [info]
            for x in range(index+1, len(infos)):
                actor = infos[x].strip()
                if actor.startswith("◎"):
                    break
                actors.append(actor)
            movie["actors"] = actors
        elif info.startswith("◎简　　介"):
            info = parse_info(info, "◎简　　介")
            for x in range(index+1, len(infos)):
                profile = infos[x].strip()
                if profile.startswith("◎"):
                    break
                movie["profile"] = profile
    
    download_url = html.xpath("//td[@bgcolor='#fdfddf']/a/@href")
    # 有些电影没有下载连接
    if len(download_url) != 0:
        movie['download_url'] = download_url[0]
    else:
        movie['download_url'] = "Not Available"
    return movie

def spider():
    base_url = 'http://www.dytt8.net/html/gndy/dyzz/list_23_{}.html'
    movies = []
    for x in range(1, 10): #(1, 179)
        # 第一个for循环，是用来控制总共有178页的
        url = base_url.format(x)
        detail_urls = get_detail_urls(url)
        for detail_url in detail_urls:
            if not detail_url.endswith('index.html'): #有些类别，叫[综合电源]有两个url。第一个以index.html结尾的没用
                # 第二个for循环，是用来遍历一页中所有电影的详情
                movie = parse_detail_page(detail_url)
                movies.append(movie)
        print("Done with page {}.".format(x))
    print(movies)

if __name__ == '__main__':
    spider()

Done with page 1.
Done with page 2.
Done with page 3.
Done with page 4.
Done with page 5.
Done with page 6.
Done with page 7.
Done with page 8.
Done with page 9.
[{'title': '2018年悬疑动作《狄仁杰之四大天王》HD国语中字', 'cover': 'https://extraimage.net/images/2018/08/30/09da45ad15c0508d5ac5da8f336bf58c.jpg', 'screenshot': 'https://lookimg.com/images/2018/08/30/GbvRh.jpg', 'year': '2018', 'country': '中国/中国香港', 'category': '动作/悬疑/古装', 'imdb_rating': '6.5/10 from 356 users', 'douban_rating': '6.5/10 from 92,590 users', 'duration': '132分钟', 'director': '徐克 Hark Tsui', 'actors': ['赵又廷 Mark Chao', '冯绍峰 Shaofeng Feng', '林更新 Gengxin Lin', '阮经天 Ethan Juan', '马思纯 Sichun Ma', '刘嘉玲 Carina Lau', '盛鉴 Chien Sheng', '杨一威 Yiwei Yang', '宝日夫 Bao Ri Fu', '张傲月 Aoyue Zhang', '孙蛟龙 Jiaolong Sun', '汪汐潮 Xichao Wang', '释胜钢 Shenggang Shi', '李炳雷 Binglei Li', '许明虎 Tiger Xu'], 'profile': '狄仁杰（赵又廷 饰）大破神都龙王案后，高宗（盛鉴 饰）御赐神器亢龙锏，遭天后武则天（刘嘉玲 饰）嫉妒，天后为盗取亢龙锏陷害狄仁杰，召集了一帮会方术的“异人组”图谋不轨，并命令尉迟真金（冯绍峰 饰）带队。狄仁杰在医官沙陀忠（林更新 饰）的帮助下成功摆脱“异人组”迫害，并和尉迟真金商议和解，与此同